In [1]:
import requests
import pandas as pd
import gzip
from datetime import datetime, timedelta, date
import time
import os
import io
import pyodbc

CD2_base_url = os.environ['CD2_base_url']
CD2_client_id = os.environ['CD2_client_id']
CD2_client_secret = os.environ['CD2_client_secret']

conn_str = os.environ["Connection_SQL"]

In [2]:
tabell = "enrollments"
sist_oppdatert = (date.today() - timedelta(days=1)).isoformat() + "Z"
start_oppdatering = time.perf_counter()
with pyodbc.connect(conn_str) as conn:
    cursor = conn.cursor()
    try:
        query = """
        MERGE INTO [dbo].[akv_sist_oppdatert] AS target 
        USING (VALUES (?, ?)) AS source (tabell, sist_oppdatert) 
        ON target.[tabell] = source.[tabell]
        WHEN MATCHED THEN
            UPDATE SET target.[sist_oppdatert] = source.[sist_oppdatert]
        WHEN NOT MATCHED THEN
            INSERT ([tabell], [sist_oppdatert]) VALUES (source.[tabell], source.[sist_oppdatert]);
        """ 
        cursor.execute(query, (tabell, sist_oppdatert))
        conn.commit()
    except pyodbc.Error as e:
        print(f"Feil ved opplasting av sist oppdatert: {e}")
print(time.perf_counter() - start_oppdatering)

0.21518339999965974
